<a href="https://colab.research.google.com/github/madhusudhanrao-ppm/dbdevrel/blob/main/source-codes/colab-code/selectai-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Oracle SelectAI with Cohere - Google Colab Notebook

This notebook demonstrates how to connect to Oracle Autonomous Database using wallet-based authentication and execute SelectAI queries with Cohere integration in Google Colab.

**Prerequisites**:
- Oracle Autonomous Database instance
- Wallet files downloaded from Oracle Cloud Console in to Google Drive
- Generate Cohere API key
- Google Drive account for storing wallet files

## Section 1: Install Required Libraries and Dependencies

Install the necessary packages including oracledb for Oracle database connectivity in Google Colab.

In [ ]:
!pip install oracledb pandas -q

In [ ]:
import oracledb
import pandas as pd
import os
import json
from pathlib import Path

print("✓ oracledb installed successfully")

## Section 2: Download and Configure Oracle Wallet

Mount Google Drive and download your Oracle wallet files. Upload your wallet.zip file to Google Drive first.

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


## Section 3: Establish Connection to Oracle Autonomous Database

Create and test the database connection using wallet-based authentication.

In [ ]:
# Connection parameters
#connection_string = "indeducation_high"  # Update with your TNS alias from tnsnames.ora
username = "DEMOUSER"  # Update with your username
adminusername = "ADMIN"  # Update with your admin username
password = "YourPassword"  # Update with your password
tns_name = "indeducation_high"
wall_config_dir = "/content/drive/MyDrive/Wallet_Folder"
wall_pwd = "YourWalletPassword"
table = "MYNOTES"
col = "NOTES"
try:
    # Connect to Oracle Database
    conn = oracledb.connect(user=adminusername,
                              password=password,
                              dsn=tns_name,
                              config_dir=wall_config_dir,
                              wallet_location=wall_config_dir,
                              wallet_password=wall_pwd)

    print("✓ Successfully connected to Oracle Database as ADMIN user")
except Exception as e:
    print(f"✗ Connection failed: {e}")
    connection = None

## Section 4: Grant Privileges and Setup SelectAI (Admin Tasks)

Grant necessary privileges for SelectAI. These commands should be executed as ADMIN user.

In [ ]:
# SQL commands to execute as ADMIN
# Note: Run these manually as ADMIN user in Oracle SQL Developer or SQL*Plus

admin_sql_commands = """
-- Grant required privileges to DEMOUSER (as ADMIN)
GRANT execute ON DBMS_CLOUD TO DEMOUSER;
GRANT execute ON DBMS_CLOUD_AI TO DEMOUSER;
COMMIT;

-- Configure Network ACL for Cohere API
BEGIN
    DBMS_NETWORK_ACL_ADMIN.APPEND_HOST_ACE(
         host => 'api.cohere.ai',
         ace  => xs$ace_type(privilege_list => xs$name_list('http'),
                             principal_name => 'DEMOUSER',
                             principal_type => xs_acl.ptype_db)
   );
END;
/
COMMIT;
"""

print("Admin Setup Commands (Execute these as ADMIN user):")
print("=" * 60)
print(admin_sql_commands)
print("=" * 60)

In [ ]:
# Close database connection
if conn:
    try:
        conn.close()
        print("✓ Database connection closed")
    except Exception as e:
        print(f"✗ Error closing connection: {e}")

In [ ]:
# Connection parameters
#connection_string = "indeducation_high"  # Update with your TNS alias from tnsnames.ora
username = "DEMOUSER"  # Update with your username
adminusername = "ADMIN"  # Update with your admin username
password = "Welcome123456#"  # Update with your password
tns_name = "indeducation_high"
wall_config_dir = "/content/drive/MyDrive/Wallet_IndEducation"
wall_pwd = "welcome1"
table = "MYNOTES"
col = "NOTES"
try:
    # Connect to Oracle Database
    conn = oracledb.connect(user=username,
                              password=password,
                              dsn=tns_name,
                              config_dir=wall_config_dir,
                              wallet_location=wall_config_dir,
                              wallet_password=wall_pwd)

    print("✓ Successfully connected to Oracle Database as DEMOUSER")
except Exception as e:
    print(f"✗ Connection failed: {e}")
    connection = None

## Section 5: Create Sample Table and Setup AI Credentials

Create a sample table and configure Cohere credentials for SelectAI.

In [ ]:
# Configuration variables
cohere_api_key = '<Your-Cohere-Key>'  # Get from https://cohere.ai/
username = 'DEMOUSER'

# Step 1: Create a copy of the sample table
cursor = conn.cursor()
try:
    cursor.execute(f"""
    CREATE TABLE {username}.CUSTOMERS AS
    SELECT * FROM SH.CUSTOMERS
    """)
    connection.commit()
    print(f"✓ Sample table CUSTOMERS created for {username}")
except oracledb.DatabaseError as e:
    if 'already exists' in str(e):
        print(f"ℹ Table CUSTOMERS already exists, skipping creation")
    else:
        print(f"⚠ Error creating table: {e}")

# Step 2: Create AI Credential for Cohere
try:
    cursor.execute(f"""
    BEGIN
        DBMS_CLOUD.create_credential(
            'COHERE_CRED',
            'COHERE',
            '{cohere_api_key}'
        );
    END;
    """)
    connection.commit()
    print("✓ Cohere credential 'COHERE_CRED' created successfully")
except oracledb.DatabaseError as e:
    if 'already exists' in str(e):
        print("ℹ Credential COHERE_CRED already exists, skipping creation")
    else:
        print(f"⚠ Error creating credential: {e}")

## Section 6: Create AI Profile

Create an AI profile that defines which tables SelectAI should have access to.

In [ ]:
# Create AI Profile for Cohere
profile_json = json.dumps({
    "provider": "COHERE",
    "credential_name": "COHERE_CRED",
    "object_list": [{"owner": username, "name": "CUSTOMERS"}]
})

try:
    cursor.execute(f"""
    BEGIN
      DBMS_CLOUD_AI.create_profile(
          'COHERE',
          '{profile_json}'
        );
    END;
    """)
    connection.commit()
    print("✓ AI Profile 'COHERE' created successfully")
except oracledb.DatabaseError as e:
    if 'already exists' in str(e):
        print("ℹ Profile COHERE already exists, skipping creation")
    else:
        print(f"⚠ Error creating profile: {e}")

# Set the profile as active
try:
    cursor.execute("EXEC DBMS_CLOUD_AI.set_profile('COHERE')")
    connection.commit()
    print("✓ Profile COHERE set as active")
except oracledb.DatabaseError as e:
    print(f"⚠ Error setting profile: {e}")

## Section 8: Generate SQL Query from Natural Language

Use SelectAI to convert natural language questions into SQL queries.

In [ ]:
# Query 1: Generate SQL from natural language prompt
prompt1 = 'what is the total number of customers'

try:
    cursor.execute(f"""
    SELECT DBMS_CLOUD_AI.GENERATE(
        prompt       => '{prompt1}',
        profile_name => 'COHERE',
        action       => 'showsql'
    ) as generated_query
    FROM dual
    """)

    result = cursor.fetchone()
    generated_sql = result[0]

    print("=" * 70)
    print(f"Query 1: {prompt1}")
    print("=" * 70)
    print("Generated SQL:")
    print(generated_sql)
    print()

except oracledb.DatabaseError as e:
    print(f"✗ Error generating SQL: {e}")

## Section 9: Generate Natural Language Explanation

Get a narrative explanation of what the query does.

In [ ]:
# Get natural language explanation
try:
    cursor.execute(f"""
    SELECT DBMS_CLOUD_AI.GENERATE(
        prompt       => '{prompt1}',
        profile_name => 'COHERE',
        action       => 'narrate'
    ) as explanation
    FROM dual
    """)

    result = cursor.fetchone()
    explanation = result[0]

    print("Narrative Explanation:")
    print("-" * 70)
    print(explanation)
    print()

except oracledb.DatabaseError as e:
    print(f"✗ Error generating explanation: {e}")

## Section 10: Test with Complex Query

Generate SQL for a more complex natural language question.

In [ ]:
# Query 2: More complex query
prompt2 = 'How many customers are married?'

try:
    # Generate SQL
    cursor.execute(f"""
    SELECT DBMS_CLOUD_AI.GENERATE(
        prompt       => '{prompt2}',
        profile_name => 'COHERE',
        action       => 'showsql'
    ) as generated_query
    FROM dual
    """)

    result = cursor.fetchone()
    generated_sql_2 = result[0]

    print("=" * 70)
    print(f"Query 2: {prompt2}")
    print("=" * 70)
    print("Generated SQL:")
    print(generated_sql_2)
    print()

except oracledb.DatabaseError as e:
    print(f"✗ Error generating SQL: {e}")

## Section 11: Process and Display Query Results

Convert results to pandas DataFrame for better visualization.

In [ ]:
import pandas as pd

# Execute the generated SQL and display results
try:
    # Execute the first generated query
    print("Executing Query 1 results:")
    print("-" * 70)
    # Make sure to read the LOB object into a string before executing
    cursor.execute(generated_sql_2.read())

    # Fetch all results
    rows = cursor.fetchall()

    # Get column names
    col_names = [description[0] for description in cursor.description]

    # Create DataFrame
    df = pd.DataFrame(rows, columns=col_names)

    print(df)
    print()

    # Query statistics
    print(f"Total rows returned: {len(df)}")
    print()

except Exception as e:
    print(f"✗ Error executing query: {e}")

## Section 12: Handle Connection Errors and Cleanup

Properly close database connections and handle any errors.

In [ ]:
# Close database connection
if conn:
    try:
        conn.close()
        print("✓ Database connection closed")
    except Exception as e:
        print(f"✗ Error closing connection: {e}")

## Troubleshooting Guide

### Connection Issues
- **Wallet not found**: Ensure wallet.zip is uploaded to Google Drive at the correct path
- **TNS name error**: Verify the TNS name matches your Oracle Cloud Console wallet
- **Authentication failed**: Check username and password are correct

### SelectAI Issues
- **Cohere API errors**: Verify API key is valid and not expired
- **Permission denied**: Ensure ADMIN user granted DBMS_CLOUD_AI privileges
- **Profile not found**: Check that profile was created successfully

### API Key Security
- Never commit real Cohere API keys to repositories
- Use environment variables or Google Colab secrets for sensitive data
- Rotate keys periodically for security

## Next Steps

1. Customize the natural language prompts for your use case
2. Create additional AI profiles for different tables
3. Integrate results with pandas for data analysis
4. Build a web application with Streamlit or FastAPI

## References

- [Oracle SelectAI Documentation](https://docs.oracle.com/en/cloud/paas/autonomous-database/index.html)
- [Cohere API Documentation](https://docs.cohere.ai/)
- [oracledb Python Driver](https://python-oracledb.readthedocs.io/)
- [Oracle Cloud Wallet Setup](https://docs.oracle.com/en/cloud/paas/autonomous-database/adbsa/connect-python-thin.html)